In [1]:
import pandas as pd
import numpy as np
from os.path import join
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
paths = [
    '../output/webface-full-celeba-simple-cnn.aligned.margin32.grey.flipped.s20.model.h5.x.npy',
    '../output/webface-full-simple-cnn.aligned.margin32.s20.model.h5.x.npy',
    
    '../output/celeba-full-simple-cnn.aligned.margin32.flipped.s20.model.h5.x.npy',
]

In [3]:
# '../output/webface-full-celeba-simple-cnn.aligned.margin32.grey.flipped.s20.model.h5.x.npy',
# '../output/webface-full-simple-cnn.aligned.margin32.s20.model.h5.x.npy',
#	0.080167	0.011866	0.032167	0.001730

#	0.081167	0.013355	0.048889	0.001846

In [4]:
X = np.hstack([np.load(p) for p in paths])

In [5]:
X.shape

(6000, 30)

In [6]:
y = np.load('../output/y')
y.shape

(6000,)

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [8]:
import xgboost as xgb
params = {}
params["objective"] = "binary:logistic"
params['eval_metric'] = ['logloss', 'error']
# params["eta"] = 0.02
# params["subsample"] = 0.7
# params["min_child_weight"] = 1
# params["colsample_bytree"] = 0.7
params["max_depth"] = 4
params["seed"] = 1632

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 500, watchlist, early_stopping_rounds=50, verbose_eval=100)

[0]	train-logloss:0.504981	train-error:0.082222	valid-logloss:0.50324	valid-error:0.085
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 50 rounds.
Stopping. Best iteration:
[38]	train-logloss:0.105833	train-error:0.035	valid-logloss:0.178113	valid-error:0.058333



In [9]:
d_all = xgb.DMatrix(X, label=y)

xgb.cv(params, d_all, 500, nfold=10, metrics=['error'], shuffle=False, early_stopping_rounds=20, seed=42)

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.098500,0.013405,0.081778,0.002034
1,0.093500,0.014934,0.075555,0.001721
2,0.093000,0.011827,0.072778,0.002155
3,0.092667,0.011479,0.071037,0.001745
4,0.089333,0.014087,0.068759,0.001567
5,0.087333,0.012849,0.067389,0.002201
6,0.088500,0.013973,0.065111,0.001391
7,0.086167,0.012803,0.063315,0.001793
8,0.085667,0.012893,0.062204,0.001696
9,0.085333,0.013577,0.060722,0.001583
